In [1]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

<span style="color:green; font-size: 35px; font-weight: bold;"> 데이터프레임과 시리즈(Pandas) </span></br>


# 9절. 데이터 그룹화 및 집계

In [5]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Goupby
- cf.pivot_table

### 1)그룹화 방법1 (df.groupby(by=[그룹핑할열])

In [25]:
# 1) 그룹화 방법1
#     df.groupby(by=[그룹핑할열])[출력될 열들].기초통계함수()
#                              [출력될 열들]생략시 그룹핑할 열을 제외한 모든 열 적용
# iris_grouped = iris.groupby(iris.Species) (비추)
# iris_grouped = iris.groupby(['Species'])  (비추)
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(3).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
10,5.4,3.7,1.5,0.2,setosa
23,5.1,3.3,1.7,0.5,setosa
44,5.1,3.8,1.9,0.4,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
69,5.6,2.5,3.9,1.1,versicolor
75,6.6,3.0,4.4,1.4,versicolor
94,5.6,2.7,4.2,1.3,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
109,7.2,3.6,6.1,2.5,virginica
136,6.3,3.4,5.6,2.4,virginica
145,6.7,3.0,5.2,2.3,virginica


In [ ]:
# 1) 그룹화 방법1
#아래 두개 함수는 같은 것이지만 아래것을 더 많이 씀
#어그리게이션펑션
iris.groupby(by='Species').sum()[['Sepal.Length']] 
iris.groupby(by='Species')[['Sepal.Length']].sum()

In [27]:
iris_grouped_sum = iris.groupby('Species').median()
iris_grouped_sum.reset_index()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [36]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [32]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [38]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa')|
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


### 2)그룹화 방법2 (피벗테이블)

In [ ]:
# 2) 그룹화 방법2
#     df.pivot_table(index=[그룹핑할열],
#                   values=[함수적용해서 출력할 열들],
#                   aggfunc=함수(기본값:mean))

In [55]:
iris.groupby('Species')[['Petal.Length','Petal.Width']].mean()
iris.pivot_table(index='Species',
                 values=['Petal.Length','Petal.Width'],
                 aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [51]:
iris.groupby('Species').count() #결측치를 제외한 값의 갯수

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [52]:
iris.pivot_table(index='Species',
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [56]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [57]:
# iris로 다중열로 그룹화하기 위해 데이터 column 추가(num 필드)
iris.shape

(150, 5)

In [62]:
# [[0]*25] + [[1]*25] + [[2]*25] + [[3]*25]
import numpy as np
np.ravel([[i]*25 for i in range(6)]) #1차원 데이터로 

# [[i]*25 for i in range(6)] #2차원 [[]]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [79]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [80]:
iris.shape

(150, 6)

In [83]:
import random
# random.randint(50)
#iris.iloc[random.sample(range(149),3),0]
# iris.iloc[0,0] 

126    6.2
62     6.0
34     4.9
Name: Sepal.Length, dtype: float64

# 질문 

In [84]:
# 각 열의 임의의 데이터에 결측치를 할당
import random
print(random.sample(range(149),3))
for col in range(iris.shape[1]-2): #Species,num 제외(-2)
    iris.iloc[random.sample(range(149), col+1), col] = np.nan

# iris.iloc[random.sample(range(149),3),3] = np.nan

[136, 18, 129]


In [85]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [87]:
#그룹별 집계 (Species, num별 카운트)
iris.groupby(by=['Species','num'])[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              24           24
versicolor 2              25           24
           3              24           24
virginica  4              25           25
           5              24           25

In [88]:
iris.pivot_table(index=['Species','num'],
                 values=['Petal.Length','Petal.Width'],
                 aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              24           24
versicolor 2              25           24
           3              24           24
virginica  4              25           25
           5              24           25

In [92]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  50.000000   50.000000  49.000000
             mean    5.006000    5.936000   6.608163
             std     0.352490    0.516171   0.626111
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.300000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  50.000000   48.000000  50.000000
             mean    3.428000    2.766667   2.974000
             std     0.379064    0.319796   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.500000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  49.000000   49.000000  49.000000
             mean    1.465306    4.265306   5.567347
             std     0.173867    0.473265   0.546728
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.400000   5.600000
             75%     1.600000    4.600000   5.900000
             max     1.900000    5.100000   6.900000
Petal.Width  count  48.000000   48.000000  50.000000
             mean    0.241667    1.320833   2.026000
             std     0.100707    0.199956   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [96]:
iris.drop('num',axis=1, inplace=True)

KeyError: "['num'] not found in axis"

In [ ]:
# 독립변수(넘파이 배열)
# 종속변수 Species
# Species를 나눠서 setosa=0,versicolor=1,virginica=2 (레이블 인코딩)

In [101]:
#레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder #LabelEncoder 클래스
le = LabelEncoder() #객체 생성
iris['taget'] = le.fit_transform(iris.Species) #iris의 Species 기준으로 변환
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,taget
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 ↔ 롱포맷
- 판다스 공식 문서의 melt : https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [107]:
# 와이드 포맷 데이터 
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                   package='datasets').data
airquality

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5
...,...,...,...,...,...,...
148,30.0,193.0,6.9,70,9,26
149,NaN,145.0,13.2,77,9,27
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


## 10.1 melt를 이용한 언피벗팅(와이드 포맷 -> 롱 포맷)

In [116]:
# 데이터 구조 변경 (와이드 포맷 -> 롱 포맷)
air_melted = airquality.melt(id_vars=['Month','Day'],
               var_name='변수명',
               value_name='값')
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [117]:
#melt 연습하기(와이드 포맷 데이터:부동산 -> 롱 포맷으로 변환)
%ls C:\ai_x\download\ShareData\부동산_250213

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 C:\ai_x\download\ShareData\부동산_250213 디렉터리

2025-04-30  오후 02:44    <DIR>          .
2025-04-30  오후 02:44    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  160,907,317,248 바이트 남음


In [123]:
import pandas as pd

In [131]:
# 지역/ 연도월 / 평당분양가


In [128]:
df.columns

Index(['지역', '2013년12월', '2014년1월', '2014년2월', '2014년3월', '2014년4월', '2014년5월',
       '2014년6월', '2014년7월', '2014년8월', '2014년9월', '2014년10월', '2014년11월',
       '2014년12월', '2015년1월', '2015년2월', '2015년3월', '2015년4월', '2015년5월',
       '2015년6월', '2015년7월', '2015년8월'],
      dtype='object')

In [138]:
df = pd.read_csv(r'C:\ai_x\download\ShareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                 encoding='cp949')

new_df = df.melt(id_vars='지역',
        var_name ='연도월',
        value_name='평당분양가')
new_df.sample()

,지역,연도월,평당분양가
215,충남,2014년12월,6989


## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷)

In [140]:
# 롱포맷-> 와이드포맷
#air_melted 
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
1,5,2,Ozone,36.0
2,5,3,Ozone,12.0
3,5,4,Ozone,18.0
4,5,5,Ozone,NaN


In [147]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                                           columns='변수명',
                                           values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name=None
airquality2

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3
...,...,...,...,...,...,...
148,9,26,30.0,193.0,70.0,6.9
149,9,27,NaN,145.0,77.0,13.2
150,9,28,14.0,191.0,75.0,14.3
151,9,29,18.0,131.0,76.0,8.0


# 11절. 데이터프레임(시리즈)에 함수 적용
- apply(func, axis) : 축요소별(요소별) 함수 적용(데이터 프레임이나 시리즈에서 가능)
        처음부터 끝까지 적용 시키는것 
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능-2차원에서만 사용 가능)
        모든 요소에 다 적용 시키는 것
- map(func, list) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply

In [157]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [171]:
def get_year(datestr):
    return int(datestr.split('-')[0]) #람다식 

In [173]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [174]:
get_year('1999-05-01')

1999

In [175]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [176]:
get_month('1999-04-01')

4

In [179]:
member['Year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]),)

In [180]:
member

,Name,Age,Email,Address,Birth,Year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [ ]:
member['Month'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[1]),)

In [186]:
X = iris.iloc[:,:-2] 
#X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.844966
Sepal.Width     3.060135
Petal.Length    3.765986
Petal.Width     1.207534
dtype: float64

# 질문

In [187]:
X.iloc[0]-avg

Sepal.Length   -0.744966
Sepal.Width     0.439865
Petal.Length   -2.365986
Petal.Width    -1.007534
dtype: float64

In [188]:
#X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리만 출력
X.apply(lambda x : round(x-avg, 2),axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.74,0.44,-2.37,-1.01
1,-0.94,-0.06,-2.37,-1.01
2,-1.14,0.14,-2.47,-1.01
3,-1.24,0.04,-2.27,-1.01
4,-0.84,0.54,-2.37,-1.01
...,...,...,...,...
145,0.86,-0.06,1.43,1.09
146,0.46,-0.56,1.23,0.69
147,0.66,-0.06,1.43,0.79
148,0.36,0.34,1.63,1.09


## 11.2 applymap

In [192]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [195]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]) )
member

,Name,Age,Email,Address,Birth,Year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [206]:
#연습하기 (apply,map)
# new_df의 연도월 데이터를 연도와 월을 정수로 분리하여 컬럼을 추가하세요.
new_df['연도'] = new_df['연도월'].apply(lambda str : int(str.split('년')[0]),)
new_df
new_df['월'] = new_df['연도월'].apply(lambda str : int(str.split('년')[1].replace('월','')),)
new_df

,지역,연도월,평당분양가,월,연도
0,서울,2013년12월,18189,12,2013
1,부산,2013년12월,8111,12,2013
2,대구,2013년12월,8080,12,2013
3,인천,2013년12월,10204,12,2013
4,광주,2013년12월,6098,12,2013
...,...,...,...,...,...
352,전북,2015년8월,6580,8,2015
353,전남,2015년8월,6289,8,2015
354,경북,2015년8월,7037,8,2015
355,경남,2015년8월,7665,8,2015


In [207]:
new_df.sample()

,지역,연도월,평당분양가,월,연도
178,세종,2014년10월,8527,10,2014


In [ ]:
#2차원 데이터 프레임의 apply 이용하는 방법

In [208]:
def year_month_append(df):
    return df

In [209]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
월              12
연도           2013
Name: 0, dtype: object

In [212]:
def year_month_append(df):
    '''
    df를 받아 연도와 월 추가 후 return
    '''
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [213]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
월              12
연도           2013
Name: 0, dtype: object

In [233]:
def split_date(df):
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df

In [238]:
split_date(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [242]:
new_df = new_df.apply(split_date, axis=1)
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


## 12절. 일괄변경하기(결측치,특정값)
### 12.1 fillna (결측치 채우는 함수) : 결측치를 특정한 값으로 변경
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [244]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                   columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [246]:
# 모든 결측치에 0 넣기
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [248]:
#결측치를 이전행의 값으로 대체
df.fillna(method='ffill') 

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [251]:
#결측치를 다음값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [256]:
#결측치를 열마다 특정값으로 대체
values = {'A':85,'B':80,'C':75,'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [261]:
#결측치는 열별 평균값으로 대체
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [263]:
#결측치는 열별 중위수로 대체 
df.fillna(value=df.median(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [266]:
# 2번째 결측치까지만 대체 (결측치 기준으로 2번째)
df.fillna(value=values, limit=2)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


## 12.2 replace(to_value,new_value,inplace=T/F)
- to_value를 new_value로 변경
- to_value에 <font color='red'>**정규표현식 이용** </font>

In [167]:
member['Birth'].str.split('-')[3][1] 

'05'

In [153]:
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [152]:
#연도월 split 년,도,월 분리
new_df['연도월'].str.split('년')

0      [2013, 12월]
1      [2013, 12월]
2      [2013, 12월]
3      [2013, 12월]
4      [2013, 12월]
          ...     
352     [2015, 8월]
353     [2015, 8월]
354     [2015, 8월]
355     [2015, 8월]
356     [2015, 8월]
Name: 연도월, Length: 357, dtype: object